# AutoGluon Fit Backend Tutorial (Distributed)

## Prerequisite
Please read **AutoGluon Task Scheduler Tutorial** first, if not yet.

## Distributed Scheduler Tutorial
![Distributed Resource Manager](img/distributed_resource_manager.png)

**Create Distributed Scheduler** for 8 servers, we launch the program from master node and provide the ip addresses for other nodes. Each node has 1 GPU (`p3.xlarge`)

In [1]:
import autogluon as ag

# Make sure the master node can access the other nodes through ssh.
# Make sure AutoGluon Package is installed on the machines.
extra_node_ips = ['172.31.1.97',
                  '172.31.3.224',
                  '172.31.6.120',
                  '172.31.1.216',
                  '172.31.14.8',
                  '172.31.14.237']

# Create a distributed scheduler. If no ipaddresses are provided, 
# scheduler will only use the local resources
scheduler = ag.distributed.DistributedTaskScheduler(extra_node_ips)

# Add additional node after intialize or at anytime.
scheduler.add_remote('172.31.3.95')

[ scheduler local ] : distributed.scheduler - INFO - -----------------------------------------------
[ worker local ] : distributed.worker - INFO -       Start worker at:   tcp://172.31.8.238:44417
[ worker local ] : distributed.worker - INFO -          Listening to:   tcp://172.31.8.238:44417
[ worker local ] : distributed.worker - INFO -              bokeh at:         172.31.8.238:35770
[ worker local ] : distributed.worker - INFO - Waiting to connect to:    tcp://172.31.8.238:8781
[ worker local ] : distributed.worker - INFO - -------------------------------------------------
[ worker local ] : distributed.worker - INFO -               Threads:                          8
[ worker local ] : distributed.worker - INFO -                Memory:                   64.39 GB
[ worker local ] : distributed.worker - INFO -       Local Directory: /home/ubuntu/git/AutoGluon/tutorials/worker-alug7osq
[ worker local ] : distributed.worker - INFO - -------------------------------------------------


/home/ubuntu/anaconda3/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
/home/ubuntu/anaconda3/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.curve, Q_S_bytes
/home/ubuntu/anaconda3/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:111: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
 

[ scheduler 172.31.1.97:8782 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
[ scheduler 172.31.1.97:8782 ] :   data = yaml.load(f.read()) or {}
[ scheduler 172.31.1.97:8782 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
[ scheduler 172.31.1.97:8782 ] :   defaults = yaml.load(f)
[ scheduler 172.31.1.97:8782 ] : distributed.scheduler - INFO - -----------------------------------------------
[ scheduler 172.31.1.97:8782 ] : distributed.scheduler - INFO - Clear task state
[ scheduler 172.31.1.97:8782 ] : distributed.scheduler - INFO -   Scheduler at:    tcp://172.31.1.97:8782
[ scheduler 172.31.1.97:8782 ] : dis

[ scheduler 172.31.6.120:8784 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
[ scheduler 172.31.6.120:8784 ] :   data = yaml.load(f.read()) or {}
[ scheduler 172.31.6.120:8784 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
[ scheduler 172.31.6.120:8784 ] :   defaults = yaml.load(f)
[ scheduler 172.31.6.120:8784 ] : distributed.scheduler - INFO - -----------------------------------------------
[ scheduler 172.31.6.120:8784 ] : distributed.scheduler - INFO - Clear task state
[ worker 172.31.6.120 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() wi

[ scheduler 172.31.14.8:8786 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
[ scheduler 172.31.14.8:8786 ] :   data = yaml.load(f.read()) or {}
[ scheduler 172.31.14.8:8786 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
[ scheduler 172.31.14.8:8786 ] :   defaults = yaml.load(f)
[ scheduler 172.31.14.8:8786 ] : distributed.scheduler - INFO - -----------------------------------------------
[ worker 172.31.14.8 ] : /home/ubuntu/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg

[ worker 172.31.3.95 ] : /home/ubuntu/anaconda3/bin/python -m distributed.cli.dask_worker 172.31.3.95:8788 --no-nanny --host 172.31.3.95
[ scheduler 172.31.3.95:8788 ] : /home/ubuntu/anaconda3/bin/python -m distributed.cli.dask_scheduler --port 8788
[ scheduler 172.31.3.95:8788 ] : distributed.scheduler - INFO - -----------------------------------------------
[ worker 172.31.3.95 ] : distributed.diskutils - INFO - Found stale lock file and directory '/home/ubuntu/worker-rzpq3lsd', purging
[ worker 172.31.3.95 ] : distributed.worker - INFO -       Start worker at:    tcp://172.31.3.95:34227
[ worker 172.31.3.95 ] : distributed.worker - INFO -          Listening to:    tcp://172.31.3.95:34227
[ worker 172.31.3.95 ] : distributed.worker - INFO -              bokeh at:          172.31.3.95:34041
[ worker 172.31.3.95 ] : distributed.worker - INFO - Waiting to connect to:     tcp://172.31.3.95:8788
[ worker 172.31.3.95 ] : distributed.worker - INFO - -----------------------------------------

In [2]:
print(scheduler)

DistributedTaskScheduler(
DistributedResourceManager{
(Remote: Remote REMOTE_ID: 0, 
	<Remote: scheduler='tcp://172.31.8.238:8781' processes=1 cores=8>, Resource: NodeResourceManager(8 CPUs, 1 GPUs))
(Remote: Remote REMOTE_ID: 1, 
	<Remote: scheduler='tcp://172.31.1.97:8782' processes=1 cores=8>, Resource: NodeResourceManager(8 CPUs, 1 GPUs))
(Remote: Remote REMOTE_ID: 2, 
	<Remote: scheduler='tcp://172.31.3.224:8783' processes=1 cores=8>, Resource: NodeResourceManager(8 CPUs, 1 GPUs))
(Remote: Remote REMOTE_ID: 3, 
	<Remote: scheduler='tcp://172.31.6.120:8784' processes=1 cores=8>, Resource: NodeResourceManager(8 CPUs, 1 GPUs))
(Remote: Remote REMOTE_ID: 4, 
	<Remote: scheduler='tcp://172.31.1.216:8785' processes=1 cores=8>, Resource: NodeResourceManager(8 CPUs, 1 GPUs))
(Remote: Remote REMOTE_ID: 5, 
	<Remote: scheduler='tcp://172.31.14.8:8786' processes=1 cores=8>, Resource: NodeResourceManager(8 CPUs, 1 GPUs))
(Remote: Remote REMOTE_ID: 6, 
	<Remote: scheduler='tcp://172.31.14.237:

**Construct a fake training function**

In [3]:
import time
import socket
from autogluon.resource import DistributedResource

@ag.autogluon_method
def train_fn(args):
    print('task_id: {}, lr: {}'.format(args.task_id, args.lr))
    # wait for 1 sec
    time.sleep(1.0)

**Launch 16 Tasks**

In [4]:
import argparse
import numpy as np
from autogluon.scheduler import Task

args = argparse.ArgumentParser()
config = {'lr': ag.searcher.sample_from(
            lambda: np.power(10.0, np.random.uniform(-4, -1)))}

for i in range(16):
    resource = DistributedResource(num_cpus=2, num_gpus=1)
    task = Task(train_fn, {'args': args, 'config': config}, resource)
    scheduler.add_task(task)

[ worker 172.31.3.224 ] : task_id: 2, lr is 0.001782649715138694
[ worker 172.31.1.97 ] : task_id: 1, lr is 0.001782649715138694
[ worker 172.31.6.120 ] : task_id: 3, lr is 0.001782649715138694
[ worker 172.31.1.216 ] : task_id: 4, lr is 0.001782649715138694
[ worker 172.31.14.237 ] : task_id: 6, lr is 0.001782649715138694
[ worker 172.31.14.8 ] : task_id: 5, lr is 0.001782649715138694
[ worker 172.31.3.95 ] : task_id: 7, lr is 0.001782649715138694
[ worker 172.31.3.95 ] : task_id: 15, lr is 0.00203228544324115
[ worker 172.31.3.224 ] : task_id: 11, lr is 0.00203228544324115
[ worker 172.31.1.97 ] : task_id: 9, lr is 0.00203228544324115
[ worker 172.31.6.120 ] : task_id: 10, lr is 0.00203228544324115
[ worker 172.31.1.216 ] : task_id: 13, lr is 0.00203228544324115
[ worker 172.31.14.237 ] : task_id: 14, lr is 0.00203228544324115
[ worker 172.31.14.8 ] : task_id: 12, lr is 0.00203228544324115
